In [ ]:
from csv import reader, writer, QUOTE_MINIMAL
from functools import reduce
from itertools import chain, cycle

from IPython.display import HTML, display
import tabulate

In [ ]:
# columns in csv sheet

column_name = 1
column_time_python_format = 13
column_time_google_format = 6

column_self_evaluation = 4
column_computed_score = 5
column_dates = 2
column_gender = 9

column_role = 6

In [ ]:
many_years = 10


def load_data(filepath="Spring_2020_League_Entries.csv"):
    print("Loading file")
    entries = []
    with open(filepath) as csv_file:
        csv = reader(csv_file, delimiter=',')
        csv = iter(csv)
        next(csv)
        for row in csv:
            try:
                time_playing = float(row[column_time_python_format])
            except ValueError:
                print("For " + row[column_name] + " changed " + row[column_time_python_format] + " to " + str(many_years))
                time_playing = many_years
            score = float(str(row[column_computed_score]).replace(",","."))

            entries.append((str(row[column_name]), str(row[column_dates]).split(', '), int(row[column_self_evaluation]),
                            str(row[column_gender]), time_playing, score, str(row[column_role])))
            #string = ""
            # for idx, field in enumerate(row):
            #     #print(field)
            #     if idx == 1 or idx == 2 or idx == 4 or idx == 6 or idx == 10:
            #         string = string + " |" + field
            # print(string)
    return entries, len(entries)


def get_score(power, time_playing):
    return (power + time_playing*2)/20

def get_avg_team_power(team):
    team_power = 0
    for member in team:
        team_power += member[2]  
    return team_power

def get_avg_team_strength(team):
    team_strength = 0
    for member in team:
        team_strength += member[5]
    return team_strength / len(team)


def save_data(teams, path="Spring_2020_League_Teams.csv"):
    with open(path, 'w', newline='') as csv_file:
        csv = writer(csv_file, delimiter=',', quotechar='"', quoting=QUOTE_MINIMAL)
        csv.writerow(['Name', 'Appearances', 'Power', 'Sex', 'Time', 'Score', 'Handler/Cutter'])
        for idx, team in enumerate(teams):
            csv.writerow(['Team' + str(idx), '---', '---', '---', '---', '---', '---'])
            #team_strength, team_power = 0, 0
            for member in team:
                #csv.writerow([member[0], member[1], member[2], member[3], member[4], member[5]])
                #team_power += member[2]
                #team_strength += member[5]
                csv.writerow(member)
            team_strength = get_avg_team_strength(team)
            team_power = get_avg_team_power(team)
            print('Team ' + str(idx) + " - Member " + str(len(team)))
            print('Avg Strength: ' + str(team_strength))
            print("Avg Power: " + str(team_power))
            #print("09.03: " + str(reduce(lambda member_x, member_y: "09.03" in member_x[1] + "09.03." in member_y[1], team)))

In [ ]:
def load_and_generate_teams():
    entries, amount_of_entries = load_data()
    w_entries, m_entries = get_sorted_list(entries)
    teams = generate_teams([w_entries, m_entries])
    return teams

def print_teams(teams):
    table = []
    for idx, team in enumerate(teams):
        table.append(["Team " + str(idx)])
        table[len(table)-1] +=[m[0] for m in team]
    table = [*zip(*table)]
        
    display(HTML(tabulate.tabulate(table, tablefmt='html')))

def get_sorted_list(entries):
    entries.sort(key=lambda entry: entry[5], reverse=True)
    entries.sort(key=lambda entry: len(entry[1]), reverse=True)
    w_entries = filter(lambda entry: entry[3] == 'w', entries)
    m_entries = filter(lambda entry: entry[3] == 'm', entries)
    return w_entries, m_entries


def generate_teams(entry_lists, team_count=6):
    teams = [[] for _ in range(team_count)]
    number_gen = cycle_up_and_down(0, team_count-1)
    for entries in entry_lists:
        for entry in entries:
            #print(str(idx) + ":" + entry[0] + ":" + str(entry[2]) + ":" + str(entry[1].count(',')+1))
            teams[next(number_gen)].append(entry)
    return teams


def cycle_up_and_down(first, last):
    up = range(first, last+1, 1)
    down = range(last, first-1, -1)
    return cycle(chain(up, down))

In [ ]:
teams = load_and_generate_teams()
print_teams(teams)
save_data(teams)